In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.python.client import device_lib

tf.logging.set_verbosity(tf.logging.WARN)

/home/robsr/miniconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

f = open("train.pickle", 'rb')
full_data = pickle.load(f)
f.close()

In [3]:
# Model Description
sense_word = 'hard'
model_name = 'basic2'
model_dir = 'output/' + sense_word + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [4]:
# Parameters
mode = 'train'
num_senses = 3
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.01
decay_steps = 500
decay_rate = 0.96
clip_norm = 1
clipping = True

In [5]:
# MODEL
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h1 = tf.concat([fw_h1, bw_h1], 2)
    
with tf.variable_scope("lstm2"):
    cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
    d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)
    
    (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
    h = tf.concat([fw_h2, bw_h2], 2)

def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.get_variable("W_att", shape = [2*hidden_size, 1], initializer = tf.contrib.layers.xavier_initializer(uniform=False, seed=0, dtype=tf.float32))
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)

with tf.variable_scope("fc_layer"):
    W_fc = tf.get_variable("W_fc", shape = [2*hidden_size, 2*hidden_size], initializer = tf.contrib.layers.xavier_initializer(uniform=False, seed=0, dtype=tf.float32))
    b_fc = tf.Variable(tf.zeros([2*hidden_size]), name="b_fc")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W_fc) + b_fc
    predictions = tf.argmax(logits, 1)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [6]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [7]:
# Splitting
data_x = full_data[sense_word][0]
data_y = full_data[sense_word][2]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.8, shuffle=True, stratify=data_y, random_state=0)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, shuffle=True, stratify=y_train, random_state=0)

len(x_train), len(x_val)

/home/robsr/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(2772, 694)

In [8]:
def data_prepare(x):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)

    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            
    return xx, xx_mask

def eval_score(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches], pred)
    return f1*100, accu*100

def model(xx, yy, mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {x:xx[s:e], y:yy[s:e], x_mask:mask[s:e], is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)
            if step%5 == 0:
                print("Steps:{}".format(step), ", Loss: {}".format(_loss))
        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [9]:
x_id_train, mask_train = data_prepare(x_train)
x_id_val, mask_val = data_prepare(x_val)
x_id_test, mask_test = data_prepare(x_test)

In [10]:
num_epochs = 30  #number of epochs
y_train = np.array(y_train)
for i in range(num_epochs):
    
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")
    
    if((i+1)%2==0):
        train_loss, train_pred = model(x_id_train, y_train, mask_train, train_cond=False)
        f1_, accu_ = eval_score(y_train, train_pred)
        print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
        f1_, accu_ = eval_score(y_val, val_pred)
        print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
test_loss, test_pred = model(x_id_test, y_test, mask_test, train_cond=False)
f1_, accu_ = eval_score(y_test, test_pred)
print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Steps:5 , Loss: 6.258545875549316
Steps:10 , Loss: 3.520009994506836
Steps:15 , Loss: 2.5930826663970947
Steps:20 , Loss: 1.8873353004455566
Steps:25 , Loss: 1.9703603982925415
Steps:30 , Loss: 1.5365471839904785
Steps:35 , Loss: 1.5063180923461914
Steps:40 , Loss: 1.4787731170654297
Epoch: 1 Step: 43 loss: 4.24284016254
Steps:45 , Loss: 1.9885308742523193
Steps:50 , Loss: 1.308830738067627
Steps:55 , Loss: 1.0597845315933228
Steps:60 , Loss: 1.0756244659423828
Steps:65 , Loss: 0.9970892667770386
Steps:70 , Loss: 0.9558264017105103
Steps:75 , Loss: 0.9669712781906128
Steps:80 , Loss: 0.9953579902648926
Steps:85 , Loss: 0.9534716606140137
Epoch: 2 Step: 86 loss: 1.11134817988
Train: F1 Score:  34.1266621591 Accuracy:  80.3779069767 Loss:  0.969300739987


/home/robsr/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Val: F1 Score:  31.9478206937 Accuracy:  80.15625 Loss:  0.954839593172
Steps:90 , Loss: 0.9774026870727539
Steps:95 , Loss: 0.778002142906189
Steps:100 , Loss: 0.9419623613357544
Steps:105 , Loss: 1.215635895729065
Steps:110 , Loss: 0.8069422245025635
Steps:115 , Loss: 0.7736497521400452
Steps:120 , Loss: 0.7267675399780273
Steps:125 , Loss: 0.7702867388725281
Epoch: 3 Step: 129 loss: 0.860231288644
Steps:130 , Loss: 0.7471674680709839
Steps:135 , Loss: 0.7986025214195251
Steps:140 , Loss: 1.336484432220459
Steps:145 , Loss: 1.0848180055618286
Steps:150 , Loss: 0.6263355016708374
Steps:155 , Loss: 0.7151857614517212
Steps:160 , Loss: 0.8212499618530273
Steps:165 , Loss: 0.6217370629310608
Steps:170 , Loss: 0.4638786315917969
Epoch: 4 Step: 172 loss: 0.793579295624
Train: F1 Score:  52.7082053696 Accuracy:  83.1395348837 Loss:  0.72580055442
Val: F1 Score:  51.6086445854 Accuracy:  82.03125 Loss:  0.789165437222
Steps:175 , Loss: 0.7286161184310913
Steps:180 , Loss: 0.6882705688476562


Steps:905 , Loss: 0.6080694198608398
Steps:910 , Loss: 0.48801520466804504
Steps:915 , Loss: 0.39554184675216675
Steps:920 , Loss: 0.44944870471954346
Steps:925 , Loss: 0.4250084459781647
Steps:930 , Loss: 0.7480498552322388
Steps:935 , Loss: 0.5143837928771973
Steps:940 , Loss: 0.5468459129333496
Steps:945 , Loss: 0.5343085527420044
Epoch: 22 Step: 946 loss: 0.569423730983
Train: F1 Score:  84.0833278948 Accuracy:  91.6424418605 Loss:  0.540815418543
Val: F1 Score:  81.6788305042 Accuracy:  90.3125 Loss:  0.601367989182
Steps:950 , Loss: 0.6205223798751831
Steps:955 , Loss: 0.4968891143798828
Steps:960 , Loss: 0.6018908023834229
Steps:965 , Loss: 0.4586239159107208
Steps:970 , Loss: 0.6373753547668457
Steps:975 , Loss: 0.5906180739402771
Steps:980 , Loss: 0.7135084271430969
Steps:985 , Loss: 0.4305490553379059
Epoch: 23 Step: 989 loss: 0.573463190434
Steps:990 , Loss: 0.4952346086502075
Steps:995 , Loss: 0.4546425938606262
Steps:1000 , Loss: 0.5113011598587036
Steps:1005 , Loss: 0.558